# Metaprogramming in Julia

In [192]:
# Drzewo składni abstrakcyjnej (AST - abstract syntax tree)
ex3 = parse("2+a*b-c")
dump(ex3)

Expr
  head: Symbol call
  args: Array{Any}((3,))
    1: Symbol -
    2: Expr
      head: Symbol call
      args: Array{Any}((3,))
        1: Symbol +
        2: Int64 2
        3: Expr
          head: Symbol call
          args: Array{Any}((3,))
            1: Symbol *
            2: Symbol a
            3: Symbol b
          typ: Any
      typ: Any
    3: Symbol c
  typ: Any


In [193]:
macro mean(arr)
    ex = Expr(:call, :/, 1, arr[1])
    for i = 2:length(arr)
         ex = Expr(:call, :+, ex, Expr(:call, :/, 1, arr[i]))
    end
    
    Expr(:call, :/, length(arr), ex)
end

@mean (macro with 4 methods)

In [194]:
@mean(2,2,5,7)

2.9787234042553195

In [195]:
macroexpand(:(@mean(2,2,5,7)))

:(4 / (((1 / 2 + 1 / 2) + 1 / 5) + 1 / 7))

### Automatic differentiation

In [196]:
function autodiff(ex::Expr)::Expr
    ???
end

LoadError: [91msyntax: colon expected in "?" expression[39m